In [3]:
from jupyter_cadquery import show, open_viewer, set_defaults
import cadquery as cq
from build123d import *
cv = open_viewer("Build123d", cad_width=770, glass=True)
set_defaults(edge_accuracy=0.0001)

In [94]:
# %load boxes_on_faces.py
"""

name: boxes_on_faces.py
by:   Gumyr
date: March 6th 2023

desc: Demo adding features to multiple faces in one operation.

license:

    Copyright 2023 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.

"""
import build123d as bd

with bd.BuildPart() as bp:
    bd.Box(3, 3, 3)
    with bd.BuildSketch(*bp.faces()):
        bd.Rectangle(1, 2, rotation=45)
    bd.extrude(amount=0.1)

assert abs(bp.part.volume - (3**3 + 6 * (1 * 2 * 0.1)) < 1e-3)

if "show_object" in locals():
    show_object(bp.part.wrapped, name="box on faces")


In [95]:
print(bp.part.volume)
bp

28.200000000000035


In [96]:
# %load build123d_customizable_logo.py
"""

name: build123d_customizable_logo.py
by:   Gumyr and modified by jdegenstein
date: December 19th 2022

desc:

    This example creates the build123d customizable logo.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

with BuildSketch() as logo_text:
    Text("123d", font_size=10, align=(Align.MIN, Align.MIN))
    font_height = logo_text.vertices().sort_by(Axis.Y)[-1].Y

with BuildSketch() as build_text:
    Text("build", font_size=5, align=(Align.CENTER, Align.CENTER))
    build_bb = bounding_box(build_text.sketch, mode=Mode.PRIVATE)
    build_vertices = build_bb.vertices().sort_by(Axis.X)
    build_width = build_vertices[-1].X - build_vertices[0].X

with BuildSketch() as cust_text:
    Text(
        "customizable",
        font_size=2.9,
        align=(Align.CENTER, Align.CENTER),
        font_style=FontStyle.BOLD,
    )
    cust_bb = bounding_box(cust_text.sketch, mode=Mode.PRIVATE)
    cust_vertices = cust_text.vertices().sort_by(Axis.X)
    cust_width = cust_vertices[-1].X - cust_vertices[0].X

with BuildLine() as one:
    l1 = Line((font_height * 0.3, 0), (font_height * 0.3, font_height))
    TangentArc(l1 @ 1, (0, font_height * 0.7), tangent=(l1 % 1) * -1)

with BuildSketch() as two:
    with Locations((font_height * 0.35, 0)):
        Text("2", font_size=10, align=(Align.MIN, Align.MIN))

with BuildPart() as three_d:
    with Locations((font_height * 1.1, 0)):
        with BuildSketch():
            Text("3d", font_size=10, align=(Align.MIN, Align.MIN))
        extrude(amount=font_height * 0.3)
        logo_width = three_d.vertices().sort_by(Axis.X)[-1].X

with BuildLine() as arrow_left:
    t1 = TangentArc((0, 0), (1, 0.75), tangent=(1, 0))
    mirror(t1, Plane.XZ)

ext_line_length = font_height * 0.5
dim_line_length = (logo_width - build_width - 2 * font_height * 0.05) / 2
with BuildLine() as extension_lines:
    l1 = Line((0, -font_height * 0.1), (0, -ext_line_length - font_height * 0.1))
    l2 = Line(
        (logo_width, -font_height * 0.1),
        (logo_width, -ext_line_length - font_height * 0.1),
    )
    with Locations(l1 @ 0.5):
        add(arrow_left.line)
    with Locations(l2 @ 0.5):
        add(arrow_left.line, rotation=180.0)
    Line(l1 @ 0.5, l1 @ 0.5 + Vector(dim_line_length, 0))
    Line(l2 @ 0.5, l2 @ 0.5 - Vector(dim_line_length, 0))

# Precisely center the build Faces
with BuildSketch() as build:
    with Locations(
        (l1 @ 0.5 + l2 @ 0.5) / 2
        - Vector((build_vertices[-1].X + build_vertices[0].X) / 2, 0)
    ):
        add(build_text.sketch)
    # add the customizable text to the build text sketch
    with Locations(
        (l1 @ 1 + l2 @ 1) / 2 - Vector((cust_vertices[-1].X + cust_vertices[0].X), 1.4)
    ):
        add(cust_text.sketch)

cmpd = Compound.make_compound(
    [three_d.part, two.sketch, one.line, build.sketch, extension_lines.line]
)

cmpd.export_svg(
    "cmpd.svg",
    (-10, 10, 60),
    (0, 0, 1),
    svg_opts={
        "pixel_scale": 20,
        "show_axes": False,
        "show_hidden": False,
    },
)

if "show_object" in locals():
    show_object(cmpd, name="compound")
    # show_object(one.line.wrapped, name="one")
    # show_object(two.sketch.wrapped, name="two")
    # show_object(three_d.part.wrapped, name="three_d")
    # show_object(extension_lines.line.wrapped, name="extension_lines")
    # show_object(build.sketch.wrapped, name="build")


In [97]:
one

In [98]:
build

In [99]:
cmpd

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  0.11s


In [100]:
# %load build123d_logo.py
"""

name: build123d_logo.py
by:   Gumyr
date: August 5th 2022

desc:

    This example creates the build123d logo.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

with BuildSketch() as logo_text:
    Text("123d", font_size=10, align=(Align.MIN, Align.MIN))
    font_height = logo_text.vertices().sort_by(Axis.Y)[-1].Y

with BuildSketch() as build_text:
    Text("build", font_size=5, align=(Align.CENTER, Align.CENTER))
    build_bb = bounding_box(build_text.sketch, mode=Mode.PRIVATE)
    build_vertices = build_bb.vertices().sort_by(Axis.X)
    build_width = build_vertices[-1].X - build_vertices[0].X

with BuildLine() as one:
    l1 = Line((font_height * 0.3, 0), (font_height * 0.3, font_height))
    TangentArc(l1 @ 1, (0, font_height * 0.7), tangent=(l1 % 1) * -1)

with BuildSketch() as two:
    with Locations((font_height * 0.35, 0)):
        Text("2", font_size=10, align=(Align.MIN, Align.MIN))

with BuildPart() as three_d:
    with Locations((font_height * 1.1, 0)):
        with BuildSketch():
            Text("3d", font_size=10, align=(Align.MIN, Align.MIN))
        extrude(amount=font_height * 0.3)
        logo_width = three_d.vertices().sort_by(Axis.X)[-1].X

with BuildLine() as arrow_left:
    t1 = TangentArc((0, 0), (1, 0.75), tangent=(1, 0))
    mirror(t1, Plane.XZ)

ext_line_length = font_height * 0.5
dim_line_length = (logo_width - build_width - 2 * font_height * 0.05) / 2
with BuildLine() as extension_lines:
    l1 = Line((0, -font_height * 0.1), (0, -ext_line_length - font_height * 0.1))
    l2 = Line(
        (logo_width, -font_height * 0.1),
        (logo_width, -ext_line_length - font_height * 0.1),
    )
    with Locations(l1 @ 0.5):
        add(arrow_left.line)
    with Locations(l2 @ 0.5):
        add(arrow_left.line, rotation=180.0)
    Line(l1 @ 0.5, l1 @ 0.5 + Vector(dim_line_length, 0))
    Line(l2 @ 0.5, l2 @ 0.5 - Vector(dim_line_length, 0))

# Precisely center the build Faces
with BuildSketch() as build:
    with Locations(
        (l1 @ 0.5 + l2 @ 0.5) / 2
        - Vector((build_vertices[-1].X + build_vertices[0].X) / 2, 0)
    ):
        add(build_text.sketch)

if False:
    logo.save("logo.step")
    exporters.export(
        logo.toCompound(),
        "logo.svg",
        opt={
            # "width": 300,
            # "height": 300,
            # "marginLeft": 10,
            # "marginTop": 10,
            "showAxes": False,
            # "projectionDir": (0.5, 0.5, 0.5),
            "strokeWidth": 0.1,
            # "strokeColor": (255, 0, 0),
            # "hiddenColor": (0, 0, 255),
            "showHidden": False,
        },
    )

if "show_object" in locals():
    show_object(one.line.wrapped, name="one")
    show_object(two.sketch.wrapped, name="two")
    show_object(three_d.part.wrapped, name="three_d")
    show_object(extension_lines.line.wrapped, name="extension_lines")
    show_object(build.sketch.wrapped, name="build")


In [101]:
# %load canadian_flag.py
"""

Projection Examples: Canadian Flag in the Wind

name: canadian_flag.py
by:   Gumyr
date: February 23th 2023

desc: A Canadian Flag blowing in the wind created by projecting planar
      faces onto a non-planar face (the_wind).

      This example also demonstrates building complex lines that snap to
      existing features.

license:

    Copyright 2023 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.

"""
from math import sin, cos, pi
from build123d import *

# Canadian Flags have a 2:1 aspect ratio
height = 50
width = 2 * height
wave_amplitude = 3


def surface(amplitude, u, v):
    """Calculate the surface displacement of the flag at a given position"""
    return v * amplitude / 20 * cos(3.5 * pi * u) + amplitude / 10 * v * sin(
        1.1 * pi * v
    )


# Note that the surface to project on must be a little larger than the faces
# being projected onto it to create valid projected faces
the_wind = Face.make_surface_from_array_of_points(
    [
        [
            Vector(
                width * (v * 1.1 / 40 - 0.05),
                height * (u * 1.2 / 40 - 0.1),
                height * surface(wave_amplitude, u / 40, v / 40) / 2,
            )
            for u in range(41)
        ]
        for v in range(41)
    ]
)
with BuildSketch(Plane.XY.offset(10)) as west_field_builder:
    Rectangle(width / 4, height, align=(Align.MIN, Align.MIN))
west_field_planar = west_field_builder.sketch.faces()[0]
east_field_planar = west_field_planar.mirror(Plane.YZ.offset(width / 2))

with BuildSketch(Plane((width / 2, 0, 10))) as center_field_builder:
    Rectangle(width / 2, height, align=(Align.CENTER, Align.MIN))
    with BuildSketch(mode=Mode.SUBTRACT) as maple_leaf_builder:
        with BuildLine() as outline:
            l1 = Polyline((0.0000, 0.0771), (0.0187, 0.0771), (0.0094, 0.2569))
            l2 = Polyline((0.0325, 0.2773), (0.2115, 0.2458), (0.1873, 0.3125))
            RadiusArc(l1 @ 1, l2 @ 0, 0.0271)
            l3 = Polyline((0.1915, 0.3277), (0.3875, 0.4865), (0.3433, 0.5071))
            TangentArc(l2 @ 1, l3 @ 0, tangent=l2 % 1)
            l4 = Polyline((0.3362, 0.5235), (0.375, 0.6427), (0.2621, 0.6188))
            SagittaArc(l3 @ 1, l4 @ 0, 0.003)
            l5 = Polyline((0.2469, 0.6267), (0.225, 0.6781), (0.1369, 0.5835))
            ThreePointArc(
                l4 @ 1, (l4 @ 1 + l5 @ 0) * 0.5 + Vector(-0.002, -0.002), l5 @ 0
            )
            l6 = Polyline((0.1138, 0.5954), (0.1562, 0.8146), (0.0881, 0.7752))
            Spline(
                l5 @ 1,
                l6 @ 0,
                tangents=(l5 % 1, l6 % 0),
                tangent_scalars=(2, 2),
            )
            l7 = Line((0.0692, 0.7808), (0.0000, 0.9167))
            TangentArc(l6 @ 1, l7 @ 0, tangent=l6 % 1)
            mirror(outline.edges(), Plane.YZ)
        make_face()
        scale(by=height)
maple_leaf_planar = maple_leaf_builder.sketch.faces()[0]
center_field_planar = center_field_builder.sketch.faces()[0]

west_field = west_field_planar.project_to_shape(the_wind, (0, 0, -1))[0]
east_field = east_field_planar.project_to_shape(the_wind, (0, 0, -1))[0]
center_field = center_field_planar.project_to_shape(the_wind, (0, 0, -1))[0]
maple_leaf = maple_leaf_planar.project_to_shape(the_wind, (0, 0, -1))[0]

if "show_object" in locals():
    # show_object(
    #     the_wind,
    #     name="the_wind",
    #     options={"alpha": 0.8, "color": (170 / 255, 85 / 255, 255 / 255)},
    # )
    show_object(west_field, name="west", options={"color": (255, 0, 0)})
    show_object(east_field, name="east", options={"color": (255, 0, 0)})
    show_object(center_field, name="center", options={"color": (255, 255, 255)})
    show_object(maple_leaf, name="maple", options={"color": (255, 0, 0)})


In [102]:
maple_leaf_planar

In [103]:
# %load circuit_board.py
"""

name: circuit_board.py
by:   Gumyr
date: September 1st 2022

desc:

    This example demonstrates placing holes around a part.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

with BuildPart() as pcb:
    with BuildSketch():
        Rectangle(70, 30)
        for i in range(65 // 5):
            x = i * 5 - 30
            with Locations((x, -15), (x, -10), (x, 10), (x, 15)):
                Circle(1, mode=Mode.SUBTRACT)
        for i in range(30 // 5 - 1):
            y = i * 5 - 10
            with Locations((30, y), (35, y)):
                Circle(1, mode=Mode.SUBTRACT)
        with GridLocations(60, 20, 2, 2):
            Circle(2, mode=Mode.SUBTRACT)
    extrude(amount=3)

if "show_object" in locals():
    show_object(pcb.part.wrapped)


In [104]:
# %load clock.py
"""

name: clock.py
by:   Gumyr
date: July 15th 2022

desc:

    This example demonstrates using polar coordinates in a sketch.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

clock_radius = 10
with BuildSketch() as minute_indicator:
    with BuildLine() as outline:
        l1 = CenterArc((0, 0), clock_radius * 0.975, 0.75, 4.5)
        l2 = CenterArc((0, 0), clock_radius * 0.925, 0.75, 4.5)
        Line(l1 @ 0, l2 @ 0)
        Line(l1 @ 1, l2 @ 1)
    make_face()
    fillet(minute_indicator.vertices(), radius=clock_radius * 0.01)

with BuildSketch() as clock_face:
    Circle(clock_radius)
    with PolarLocations(0, 60):
        add(minute_indicator.sketch, mode=Mode.SUBTRACT)
    with PolarLocations(clock_radius * 0.875, 12):
        SlotOverall(clock_radius * 0.05, clock_radius * 0.025, mode=Mode.SUBTRACT)
    for hour in range(1, 13):
        with PolarLocations(clock_radius * 0.75, 1, -hour * 30 + 90, 360, rotate=False):
            Text(
                str(hour),
                font_size=clock_radius * 0.175,
                font_style=FontStyle.BOLD,
                mode=Mode.SUBTRACT,
            )

if "show_object" in locals():
    show_object(clock_face.sketch.wrapped, name="clock_face")


In [3]:
# %load custom_sketch_objects.py
"""

name: custom_sketch_objects.py
by:   Gumyr
date: Jan 21st 2023

desc:

    This example demonstrates the creation of a Playing Card storage box with
    user generated custom BuildSketch objects. Four new BuildSketch objects are
    created: Club, Spade, Heart, and Diamond, which are then used to punch
    holes into the top of the box's lid.

license:

    Copyright 2023 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *


class Club(BaseSketchObject):
    """Sketch Object: Club

    The club suit symbol from a playing card.

    Args:
        height (float): size along the Y-axis
        rotation (float, optional): angle from X-axis. Defaults to 0.
        align (tuple[Align, Align], optional): align min, center, or max of object.
            Defaults to (Align.CENTER, Align.CENTER).
        mode (Mode, optional): combination mode. Defaults to Mode.ADD.
    """

    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        # Create the club shape
        # Note: The workplane and mode must be set here to avoid interactions with
        #       builders in difference scopes.
        with BuildSketch(Plane.XY, mode=Mode.PRIVATE) as club:
            with BuildLine():
                l0 = Line((0, -188), (76, -188))
                b0 = Bezier(l0 @ 1, (61, -185), (33, -173), (17, -81))
                b1 = Bezier(b0 @ 1, (49, -128), (146, -145), (167, -67))
                b2 = Bezier(b1 @ 1, (187, 9), (94, 52), (32, 18))
                b3 = Bezier(b2 @ 1, (92, 57), (113, 188), (0, 188))
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / club.sketch.bounding_box().size.Y)

        # Pass the shape to the BaseSketchObject class to create a new Club object
        super().__init__(obj=club.sketch, rotation=rotation, align=align, mode=mode)


class Spade(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch(Plane.XY, mode=Mode.PRIVATE) as spade:
            with BuildLine():
                b0 = Bezier((0, 198), (6, 190), (41, 127), (112, 61))
                b1 = Bezier(b0 @ 1, (242, -72), (114, -168), (11, -105))
                b2 = Bezier(b1 @ 1, (31, -174), (42, -179), (53, -198))
                l0 = Line(b2 @ 1, (0, -198))
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / spade.sketch.bounding_box().size.Y)
        super().__init__(obj=spade.sketch, rotation=rotation, align=align, mode=mode)


class Heart(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch(Plane.XY, mode=Mode.PRIVATE) as heart:
            with BuildLine():
                b1 = Bezier((0, 146), (20, 169), (67, 198), (97, 198))
                b2 = Bezier(b1 @ 1, (125, 198), (151, 186), (168, 167))
                b3 = Bezier(b2 @ 1, (197, 133), (194, 88), (158, 31))
                b4 = Bezier(b3 @ 1, (126, -13), (94, -48), (62, -95))
                b5 = Bezier(b4 @ 1, (40, -128), (0, -198))
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / heart.sketch.bounding_box().size.Y)
        super().__init__(obj=heart.sketch, rotation=rotation, align=align, mode=mode)


class Diamond(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch(Plane.XY, mode=Mode.PRIVATE) as diamond:
            with BuildLine():
                Bezier((135, 0), (94, 69), (47, 134), (0, 198))
                mirror(about=Plane.XZ)
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / diamond.sketch.bounding_box().size.Y)
        super().__init__(obj=diamond.sketch, rotation=rotation, align=align, mode=mode)


# The inside of the box fits 2.5x3.5" playing card deck with a small gap
pocket_w = 2.5 * IN + 2 * MM
pocket_l = 3.5 * IN + 2 * MM
pocket_t = 0.5 * IN + 2 * MM
wall_t = 3 * MM  # Wall thickness
bottom_t = wall_t / 2  # Top and bottom thickness
lid_gap = 0.5 * MM  # Spacing between base and lid
lip_t = wall_t / 2 - lid_gap / 2  # Lip thickness


with BuildPart() as box_builder:
    with BuildSketch() as box_plan:
        RectangleRounded(pocket_w + 2 * wall_t, pocket_l + 2 * wall_t, pocket_w / 15)
    extrude(amount=bottom_t + pocket_t / 2)
    base_top = box_builder.faces().sort_by(Axis.Z)[-1]
    with BuildSketch(base_top) as walls:
        offset(box_plan.sketch, amount=-lip_t, mode=Mode.ADD)
    extrude(amount=pocket_t / 2)
    with BuildSketch(Plane.XY.offset(wall_t / 2)):
        offset(box_plan.sketch, amount=-wall_t, mode=Mode.ADD)
    extrude(amount=pocket_t, mode=Mode.SUBTRACT)
box = box_builder.part

with BuildPart() as lid_builder:
    add(box_plan.sketch)
    extrude(amount=pocket_t / 2 + bottom_t)
    with BuildSketch() as pocket:
        offset(box_plan.sketch, amount=-(wall_t - lip_t), mode=Mode.ADD)
    extrude(amount=pocket_t / 2, mode=Mode.SUBTRACT)

    with BuildSketch(lid_builder.faces().sort_by(Axis.Z)[-1]) as suits:
        with Locations((-0.3 * pocket_w, 0.3 * pocket_l)):
            Heart(pocket_l / 5)
        with Locations((-0.3 * pocket_w, -0.3 * pocket_l)):
            Diamond(pocket_l / 5)
        with Locations((0.3 * pocket_w, 0.3 * pocket_l)):
            Spade(pocket_l / 5)
        with Locations((0.3 * pocket_w, -0.3 * pocket_l)):
            Club(pocket_l / 5)
    extrude(amount=-wall_t, mode=Mode.SUBTRACT)
lid = lid_builder.part.moved(Location((0, 0, (wall_t + pocket_t) / 2)))

if "show_object" in locals():
    show_object(box, name="box")
    show_object(lid, name="lid", options={"alpha": 0.6})


In [5]:
# %load din_rail.py
"""

name: din_rail.py
by:   Gumyr
date: July 14th 2022

desc:

    This example demonstrates multiple vertex filtering techniques including
    a fully custom filter. It also shows how a workplane can be replaced
    with another in a different orientation for further work.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
import logging
from build123d import *

logging.basicConfig(
    filename="din_rail.log",
    level=logging.INFO,
    format="%(name)s-%(levelname)s %(asctime)s - [%(filename)s:%(lineno)s - %(funcName)20s() ] - %(message)s",
)
logging.info("Starting to create din rail")

# 35x7.5mm DIN Rail Dimensions
overall_width, top_width, height, thickness, fillet_radius = 35, 27, 7.5, 1, 0.8
rail_length = 1000
slot_width, slot_length, slot_pitch = 6.2, 15, 25

with BuildPart() as rail:
    with BuildSketch(Plane.XZ) as din:
        Rectangle(overall_width, thickness, align=(Align.CENTER, Align.MIN))
        Rectangle(top_width, height, align=(Align.CENTER, Align.MIN))
        Rectangle(
            top_width - 2 * thickness,
            height - thickness,
            align=(Align.CENTER, Align.MIN),
            mode=Mode.SUBTRACT,
        )
        inside_vertices = (
            din.vertices()
            .filter_by_position(Axis.Y, 0.0, height, inclusive=(False, False))
            .filter_by_position(
                Axis.X,
                -overall_width / 2,
                overall_width / 2,
                inclusive=(False, False),
            )
        )
        fillet(inside_vertices, radius=fillet_radius)
        outside_vertices = filter(
            lambda v: (v.Y == 0.0 or v.Y == height)
            and -overall_width / 2 < v.X < overall_width / 2,
            din.vertices(),
        )
        fillet(outside_vertices, radius=fillet_radius + thickness)
    extrude(amount=rail_length / 2, both=True)

    with BuildSketch(Plane.XY) as slots:
        with GridLocations(
            0,
            slot_pitch,
            1,
            rail_length // slot_pitch - 1,
        ):
            SlotOverall(slot_length, slot_width, rotation=90)
    extrude(amount=height, mode=Mode.SUBTRACT)

# assert abs(rail.part.volume - 42462.863388694714) < 1e-3
if "show_object" in locals():
    show_object(rail.part.wrapped, name="rail")


In [6]:
rail

In [108]:
# %load extrude.py
"""

name: extrude.py
by:   Gumyr
date: September 20th 2022

desc:

    This example demonstrates multiple uses of Extrude cumulating in
    the design of a key cap.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

# Extrude pending face by amount
with BuildPart() as simple:
    with BuildSketch():
        Text("O", font_size=10)
    extrude(amount=5)

# Extrude pending face in both directions by amount
with BuildPart() as both:
    with BuildSketch():
        Text("O", font_size=10)
    extrude(amount=5, both=True)

# Extrude multiple pending faces on multiple faces
with BuildPart() as multiple:
    Box(10, 10, 10)
    with BuildSketch(*multiple.faces()):
        with GridLocations(5, 5, 2, 2):
            Text("Ω", font_size=3)
    extrude(amount=1)

# Non-planar surface
with BuildPart() as non_planar:
    Cylinder(10, 20, rotation=(90, 0, 0), align=(Align.CENTER, Align.MIN, Align.CENTER))
    Box(10, 10, 10, align=(Align.CENTER, Align.CENTER, Align.MIN), mode=Mode.INTERSECT)
    extrude(non_planar.part.faces().sort_by(Axis.Z)[0], amount=2, mode=Mode.REPLACE)


rad, rev = 3, 25

# Extrude last
with BuildPart() as ex26:
    with BuildSketch() as ex26_sk:
        with Locations((0, rev)):
            Circle(rad)
    revolve(axis=Axis.X, revolution_arc=90)
    mirror(about=Plane.XZ)
    with BuildSketch() as ex26_sk2:
        Rectangle(rad, rev)
    ex26_target = ex26.part
    extrude(until=Until.LAST, clean=False, mode=Mode.REPLACE)

# Extrude next
with BuildPart() as ex27:
    with BuildSketch():
        with Locations((0, rev)):
            Circle(rad)
    revolve(axis=Axis.X, revolution_arc=90)
    with BuildSketch(Plane.XZ):
        with Locations((0, rev)):
            Circle(rad)
    revolve(axis=Axis.X, revolution_arc=150)
    with BuildSketch(Plane.XY.offset(-60)):
        Rectangle(rad, rev + 25)
    extrusion27 = extrude(until=Until.NEXT, mode=Mode.ADD)

# Extrude next both
with BuildPart() as ex28:
    Torus(25, 5, rotation=(0, 90, 0))
    with BuildSketch():
        Rectangle(rad, rev)
    extrusion28 = extrude(until=Until.NEXT, both=True)

if "show_object" in locals():
    show_object(
        simple.part.translate((-15, 0, 0)).wrapped, name="simple pending extrude"
    )
    show_object(both.part.translate((20, 10, 0)).wrapped, name="simple both")
    show_object(
        multiple.part.translate((0, -20, 0)).wrapped, name="multiple pending extrude"
    )
    show_object(non_planar.part.translate((20, -10, 0)).wrapped, name="non planar")
    show_object(
        ex26_target.translate((-40, 0, 0)).wrapped,
        name="extrude until last target",
        options={"alpha": 0.8},
    )
    show_object(
        ex26.part.translate((-40, 0, 0)).wrapped,
        name="extrude until last",
    )
    show_object(
        ex27.part.rotate(Axis.Z, 90).translate((0, 50, 0)).wrapped,
        name="extrude until next target",
        options={"alpha": 0.8},
    )
    show_object(
        extrusion27.rotate(Axis.Z, 90).translate((0, 50, 0)).wrapped,
        name="extrude until next",
    )
    show_object(
        ex28.part.rotate(Axis.Z, -90).translate((0, -50, 0)).wrapped,
        name="extrude until next both target",
        options={"alpha": 0.8},
    )
    show_object(
        extrusion28.rotate(Axis.Z, -90).translate((0, -50, 0)).wrapped,
        name="extrude until next both",
    )


In [4]:
# %load handle.py
"""

name: handle.py
by:   Gumyr
date: July 29th 2022

desc:

    This example demonstrates multisection sweep creating a drawer handle.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

segment_count = 6

with BuildPart() as handle:
    # Create a path for the sweep along the handle - added to pending_edges
    with BuildLine() as handle_center_line:
        Spline(
            (-10, 0, 0),
            (0, 0, 5),
            (10, 0, 0),
            tangents=((0, 0, 1), (0, 0, -1)),
            tangent_scalars=(1.5, 1.5),
        )
    # Record the center line for display and workplane creation
    handle_path: Wire = handle_center_line.wires()[0]

    # Create the cross sections - added to pending_faces
    for i in range(segment_count + 1):
        with BuildSketch(
            Plane(
                origin=handle_path @ (i / segment_count),
                z_dir=handle_path % (i / segment_count),
            )
        ) as section:
            if i % segment_count == 0:
                Circle(1)
            else:
                Rectangle(1.25, 3)
                fillet(section.vertices(), radius=0.2)
    # Record the sections for display
    sections = handle.pending_faces

    # Create the handle by sweeping along the path
    sweep(multisection=True)

assert abs(handle.part.volume - 94.77361455046953) < 1e-3

if "show_object" in locals():
    show_object(handle_path.wrapped, name="handle_path")
    for i, section in enumerate(sections):
        show_object(section.wrapped, name="section" + str(i))
    show_object(handle.part.wrapped, name="handle", options=dict(alpha=0.6))


In [9]:
handle

In [8]:
handle_center_line

In [111]:
# %load heat_exchanger.py
"""

name: heat_exchanger.py
by:   Gumyr
date: October 8th 2022

desc:

    This example creates a model of a parametric heat exchanger core.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

exchanger_diameter = 10 * CM
exchanger_length = 30 * CM
plate_thickness = 5 * MM
# 149 tubes
tube_diameter = 5 * MM
tube_spacing = 2 * MM
tube_wall_thickness = 0.5 * MM
tube_extension = 3 * MM
bundle_diameter = exchanger_diameter - 2 * tube_diameter
fillet_radius = tube_spacing / 3
assert tube_extension > fillet_radius

# Build the heat exchanger
with BuildPart() as heat_exchanger:
    # Generate list of tube locations
    tube_locations = [
        l
        for l in HexLocations(
            apothem=(tube_diameter + tube_spacing) / 2,
            x_count=exchanger_diameter // tube_diameter,
            y_count=exchanger_diameter // tube_diameter,
        )
        if l.position.length < bundle_diameter / 2
    ]
    tube_count = len(tube_locations)
    with BuildSketch() as tube_plan:
        with Locations(*tube_locations):
            Circle(radius=tube_diameter / 2)
            Circle(radius=tube_diameter / 2 - tube_wall_thickness, mode=Mode.SUBTRACT)
    extrude(amount=exchanger_length / 2)
    with BuildSketch(
        Plane(
            origin=(0, 0, exchanger_length / 2 - tube_extension - plate_thickness),
            z_dir=(0, 0, 1),
        )
    ) as plate_plan:
        Circle(radius=exchanger_diameter / 2)
        with Locations(*tube_locations):
            Circle(radius=tube_diameter / 2 - tube_wall_thickness, mode=Mode.SUBTRACT)
    extrude(amount=plate_thickness)
    half_volume_before_fillet = heat_exchanger.part.volume
    # Simulate welded tubes by adding a fillet to the outside radius of the tubes
    fillet(
        heat_exchanger.edges()
        .filter_by(GeomType.CIRCLE)
        .sort_by(SortBy.RADIUS)
        .sort_by(Axis.Z, reverse=True)[2 * tube_count : 3 * tube_count],
        radius=fillet_radius,
    )
    half_volume_after_fillet = heat_exchanger.part.volume
    mirror(about=Plane.XY)

fillet_volume = 2 * (half_volume_after_fillet - half_volume_before_fillet)
assert abs(fillet_volume - 469.88331045553787) < 1e-3

if "show_object" in locals():
    show_object(heat_exchanger.part.wrapped)


In [112]:
heat_exchanger

In [113]:
# %load holes.py
"""

name: holes.py
by:   Gumyr
date: September 26th 2022

desc:

    This example demonstrates multiple hole types.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

# Simple through hole
with BuildPart() as thru_hole:
    Cylinder(radius=3, height=2)
    Hole(radius=1)

# Recessed counter bore hole (hole location @ (0,0,0))
with BuildPart() as recessed_counter_bore:
    with Locations((10, 0)):
        Cylinder(radius=3, height=2)
        CounterBoreHole(radius=1, counter_bore_radius=1.5, counter_bore_depth=0.5)

# Recessed counter sink hole (hole location @ (0,0,0))
with BuildPart() as recessed_counter_sink:
    with Locations((0, 10)):
        Cylinder(radius=3, height=2)
        CounterSinkHole(radius=1, counter_sink_radius=1.5)

# Flush counter sink hole (hole location @ (0,0,2))
with BuildPart() as flush_counter_sink:
    with Locations((10, 10)):
        Cylinder(radius=3, height=2)
        with Locations(
            (0, 0, flush_counter_sink.part.faces().sort_by(Axis.Z)[-1].center().Z)
        ):
            CounterSinkHole(radius=1, counter_sink_radius=1.5)

if "show_object" in locals():
    show_object(thru_hole.part.wrapped, name="though hole")
    show_object(recessed_counter_bore.part.wrapped, name="recessed counter bore")
    show_object(recessed_counter_sink.part.wrapped, name="recessed counter sink")
    show_object(flush_counter_sink.part.wrapped, name="flush counter sink")


In [114]:
#thru_hole
#recessed_counter_bore
#recessed_counter_sink
flush_counter_sink

In [115]:
# %load intersecting_chamfers.py
from build123d import *


with BuildPart() as blocks:
    with Locations((-1, -1, 0)):
        Box(1, 2, 1, align=(Align.CENTER, Align.MIN, Align.MIN))
    Box(1, 1, 2, align=(Align.CENTER, Align.MIN, Align.MIN))
    with Locations((1, -1, 0)):
        Box(1, 2, 1, align=(Align.CENTER, Align.MIN, Align.MIN))
    bottom_edges = blocks.edges().filter_by_position(
        Axis.Z, 0, 1, inclusive=(True, False)
    )
    chamfer(bottom_edges, length=0.1)
    top_edges = blocks.edges().filter_by_position(Axis.Z, 1, 2, inclusive=(False, True))
    chamfer(top_edges, length=0.1)


if "show_object" in locals():
    show_object(blocks.part.wrapped)


In [116]:
blocks

In [117]:
# %load intersecting_pipes.py
"""

name: intersecting_pipes.py
by:   Gumyr
date: July 14th 2022

desc:

    This example demonstrates working on multiple planes created from object
    faces and using a Select.LAST selector to return edges to be filleted.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
import logging
from build123d import *

# logging.basicConfig(
#     filename="intersecting_pipes.log",
#     level=logging.DEBUG,
#     format="%(name)s-%(levelname)5s %(asctime)s - [%(filename)s:%(lineno)s - %(funcName)20s() ] - %(message)s",
# )
# logging.info("Starting pipes test")

with BuildPart() as pipes:
    box = Box(10, 10, 10, rotation=(10, 20, 30))
    with BuildSketch(*box.faces()) as pipe:
        Circle(4)
    extrude(amount=-5, mode=Mode.SUBTRACT)
    with BuildSketch(*box.faces()) as pipe:
        Circle(4.5)
        Circle(4, mode=Mode.SUBTRACT)
    extrude(amount=10)
    fillet(pipes.edges(Select.LAST), 0.2)

assert abs(pipes.part.volume - 1015.939005681509) < 1e-3

if "show_object" in locals():
    show_object(pipes.part.wrapped, name="intersecting pipes")


In [118]:
pipes

In [119]:
# %load joints.py
"""
Experimental Joint development file
"""
from build123d import *


class JointBox(Solid):
    """A filleted box with joints

    A box of the given dimensions with all of the edges filleted.

    Args:
        length (float): box length
        width (float): box width
        height (float): box height
        radius (float): edge radius
        taper (float): vertical taper in degrees
    """

    def __init__(
        self,
        length: float,
        width: float,
        height: float,
        radius: float = 0.0,
        taper: float = 0.0,
    ):
        # Create the object
        with BuildPart() as obj:
            with BuildSketch():
                Rectangle(length, width)
            extrude(amount=height, taper=taper)
            if radius != 0.0:
                fillet(obj.part.edges(), radius=radius)
            Cylinder(width / 4, length, rotation=(0, 90, 0), mode=Mode.SUBTRACT)
        # Initialize the Solid class with the new OCCT object
        super().__init__(obj.part.wrapped)


#
# Base Object
#
# base = JointBox(10, 10, 10)
# base = JointBox(10, 10, 10).locate(Location(Vector(1, 1, 1)))
# base = JointBox(10, 10, 10).locate(Location(Vector(1, 1, 1), (1, 0, 0), 5))
base: JointBox = JointBox(10, 10, 10, taper=3).locate(
    Location(Vector(1, 1, 1), (1, 1, 1), 30)
)
base_top_edges: ShapeList[Edge] = (
    base.edges().filter_by(Axis.X, tolerance=30).sort_by(Axis.Z)[-2:]
)
#
# Rigid Joint
#
fixed_arm = JointBox(1, 1, 5, 0.2)
j1 = RigidJoint("side", base, Plane(base.faces().sort_by(Axis.X)[-1]).to_location())
j2 = RigidJoint(
    "top", fixed_arm, (-Plane(fixed_arm.faces().sort_by(Axis.Z)[-1])).to_location()
)
base.joints["side"].connect_to(fixed_arm.joints["top"])
# or
# j1.connect_to(j2)

#
# Hinge
#
hinge_arm = JointBox(2, 1, 10, taper=1)
swing_arm_hinge_edge: Edge = (
    hinge_arm.edges()
    .group_by(SortBy.LENGTH)[-1]
    .sort_by(Axis.X)[-2:]
    .sort_by(Axis.Y)[0]
)
swing_arm_hinge_axis = swing_arm_hinge_edge.to_axis()
base_corner_edge = base.edges().sort_by(Axis((0, 0, 0), (1, 1, 0)))[-1]
base_hinge_axis = base_corner_edge.to_axis()
j3 = RevoluteJoint("hinge", base, axis=base_hinge_axis, angular_range=(0, 180))
j4 = RigidJoint("corner", hinge_arm, swing_arm_hinge_axis.to_location())
base.joints["hinge"].connect_to(hinge_arm.joints["corner"], angle=90)

#
# Slider
#
slider_arm = JointBox(4, 1, 2, 0.2)
s1 = LinearJoint(
    "slide",
    base,
    axis=Edge.make_mid_way(*base_top_edges, 0.67).to_axis(),
    linear_range=(0, base_top_edges[0].length),
)
s2 = RigidJoint("slide", slider_arm, Location(Vector(0, 0, 0)))
base.joints["slide"].connect_to(slider_arm.joints["slide"], position=8)
# s1.connect_to(s2,8)

#
# Cylindrical
#
hole_axis = Axis(
    base.faces().sort_by(Axis.Y)[0].center(),
    -base.faces().sort_by(Axis.Y)[0].normal_at(),
)
screw_arm = JointBox(1, 1, 10, 0.49)
j5 = CylindricalJoint("hole", base, hole_axis, linear_range=(-10, 10))
j6 = RigidJoint("screw", screw_arm, screw_arm.faces().sort_by(Axis.Z)[-1].location)
j5.connect_to(j6, position=-1, angle=90)

#
# PinSlotJoint
#
j7 = LinearJoint(
    "slot",
    base,
    axis=Edge.make_mid_way(*base_top_edges, 0.33).to_axis(),
    linear_range=(0, base_top_edges[0].length),
)
pin_arm = JointBox(2, 1, 2)
j8 = RevoluteJoint("pin", pin_arm, axis=Axis.Z, angular_range=(0, 360))
j7.connect_to(j8, position=6, angle=60)

#
# BallJoint
#
j9 = BallJoint("socket", base, Plane(base.faces().sort_by(Axis.X)[0]).to_location())
ball = JointBox(2, 2, 2, 0.99)
j10 = RigidJoint("ball", ball, Location(Vector(0, 0, 1)))
j9.connect_to(j10, angles=(10, 20, 30))

if "show_object" in locals():
    show_object(base, name="base", options={"alpha": 0.8})
    show_object(base.joints["side"].symbol, name="side joint")
    show_object(base.joints["hinge"].symbol, name="hinge joint")
    show_object(base.joints["slide"].symbol, name="slot joint")
    show_object(base.joints["slot"].symbol, name="pin slot joint")
    show_object(base.joints["hole"].symbol, name="hole")
    show_object(base.joints["socket"].symbol, name="socket joint")
    show_object(hinge_arm.joints["corner"].symbol, name="hinge_arm joint")
    show_object(fixed_arm, name="fixed_arm", options={"alpha": 0.6})
    show_object(fixed_arm.joints["top"].symbol, name="fixed_arm joint")
    show_object(hinge_arm, name="hinge_arm", options={"alpha": 0.6})
    show_object(slider_arm, name="slider_arm", options={"alpha": 0.6})
    show_object(pin_arm, name="pin_arm", options={"alpha": 0.6})
    show_object(slider_arm.joints["slide"].symbol, name="slider attachment")
    show_object(pin_arm.joints["pin"].symbol, name="pin axis")
    show_object(screw_arm, name="screw_arm")
    show_object(ball, name="ball", options={"alpha": 0.6})


In [120]:
fixed_arm

In [121]:
# %load key_cap.py
"""

name: key_cap.py
by:   Gumyr
date: September 20th 2022

desc:

    This example demonstrates the design of a Cherry MX key cap by using
    extrude with a taper and extrude until next.

    See: https://www.cherrymx.de/en/dev.html

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

with BuildPart() as key_cap:
    # Start with the plan of the key cap and extrude it
    with BuildSketch() as plan:
        Rectangle(18 * MM, 18 * MM)
    extrude(amount=10 * MM, taper=15)
    # Create a dished top
    with Locations((0, -3 * MM, 47 * MM)):
        Sphere(40 * MM, mode=Mode.SUBTRACT, rotation=(90, 0, 0))
    # Fillet all the edges except the bottom
    fillet(
        key_cap.edges().filter_by_position(Axis.Z, 0, 30 * MM, inclusive=(False, True)),
        radius=1 * MM,
    )
    # Hollow out the key by subtracting a scaled version
    scale(by=(0.925, 0.925, 0.85), mode=Mode.SUBTRACT)

    # Add supporting ribs while leaving room for switch activation
    with BuildSketch(Plane(origin=(0, 0, 4 * MM))):
        Rectangle(15 * MM, 0.5 * MM)
        Rectangle(0.5 * MM, 15 * MM)
        Circle(radius=5.5 * MM / 2)
    # Extrude the mount and ribs to the key cap underside
    extrude(until=Until.NEXT)
    # Find the face on the bottom of the ribs to build onto
    rib_bottom = key_cap.faces().filter_by_position(Axis.Z, 4 * MM, 4 * MM)[0]
    # Add the switch socket
    with BuildSketch(rib_bottom) as cruciform:
        Circle(radius=5.5 * MM / 2)
        Rectangle(4.1 * MM, 1.17 * MM, mode=Mode.SUBTRACT)
        Rectangle(1.17 * MM, 4.1 * MM, mode=Mode.SUBTRACT)
    extrude(amount=3.5 * MM, mode=Mode.ADD)

assert abs(key_cap.part.volume - 644.8900473617498) < 1e-3

if "show_object" in locals():
    show_object(key_cap.part.wrapped, name="key cap", options={"alpha": 0.7})


In [122]:
key_cap

In [123]:
# %load lego.py
"""

name: lego.py
by:   Gumyr
date: September 12th 2022

desc:

    This example creates a model of a double wide lego block with a
    parametric length (pip_count).
    *** Don't edit this file without checking the lego tutorial ***

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

pip_count = 6

lego_unit_size = 8
pip_height = 1.8
pip_diameter = 4.8
block_length = lego_unit_size * pip_count
block_width = 16
base_height = 9.6
block_height = base_height + pip_height
support_outer_diameter = 6.5
support_inner_diameter = 4.8
ridge_width = 0.6
ridge_depth = 0.3
wall_thickness = 1.2

svg_opts = {
    "pixel_scale": 20,
    "show_axes": False,
    "show_hidden": False,
}

with BuildPart() as lego:
    # Draw the bottom of the block
    with BuildSketch() as plan:
        # Start with a Rectangle the size of the block
        perimeter = Rectangle(width=block_length, height=block_width)
        plan.sketch.export_svg(
            "tutorial_step4.svg", (0, 0, 10), (0, 1, 0), svg_opts=svg_opts
        )
        # Subtract an offset to create the block walls
        offset(
            perimeter,
            -wall_thickness,
            kind=Kind.INTERSECTION,
            mode=Mode.SUBTRACT,
        )
        plan.sketch.export_svg(
            "tutorial_step5.svg", (0, 0, 10), (0, 1, 0), svg_opts=svg_opts
        )
        # Add a grid of lengthwise and widthwise bars
        with GridLocations(x_spacing=0, y_spacing=lego_unit_size, x_count=1, y_count=2):
            Rectangle(width=block_length, height=ridge_width)
        with GridLocations(lego_unit_size, 0, pip_count, 1):
            Rectangle(width=ridge_width, height=block_width)
        plan.sketch.export_svg(
            "tutorial_step6.svg", (0, 0, 10), (0, 1, 0), svg_opts=svg_opts
        )
        # Substract a rectangle leaving ribs on the block walls
        Rectangle(
            block_length - 2 * (wall_thickness + ridge_depth),
            block_width - 2 * (wall_thickness + ridge_depth),
            mode=Mode.SUBTRACT,
        )
        plan.sketch.export_svg(
            "tutorial_step7.svg", (0, 0, 10), (0, 1, 0), svg_opts=svg_opts
        )
        # Add a row of hollow circles to the center
        with GridLocations(
            x_spacing=lego_unit_size, y_spacing=0, x_count=pip_count - 1, y_count=1
        ):
            Circle(radius=support_outer_diameter / 2)
            Circle(radius=support_inner_diameter / 2, mode=Mode.SUBTRACT)
        plan.sketch.export_svg(
            "tutorial_step8.svg", (0, 0, 10), (0, 1, 0), svg_opts=svg_opts
        )
    # Extrude this base sketch to the height of the walls
    extrude(amount=base_height - wall_thickness)
    lego.part.export_svg(
        "tutorial_step9.svg", (-5, -30, 50), (0, 0, 1), svg_opts=svg_opts
    )
    # Create a box on the top of the walls
    with Locations((0, 0, lego.vertices().sort_by(Axis.Z)[-1].Z)):
        # Create the top of the block
        Box(
            length=block_length,
            width=block_width,
            height=wall_thickness,
            align=(Align.CENTER, Align.CENTER, Align.MIN),
        )
    lego.part.export_svg(
        "tutorial_step10.svg",
        (-5, -30, 50),
        (0, 0, 1),
        svg_opts={
            "pixel_scale": 20,
            "show_axes": False,
            "show_hidden": True,
        },
    )
    # Create a workplane on the top of the block
    with BuildPart(lego.faces().sort_by(Axis.Z)[-1]):
        # Create a grid of pips
        with GridLocations(lego_unit_size, lego_unit_size, pip_count, 2):
            Cylinder(
                radius=pip_diameter / 2,
                height=pip_height,
                align=(Align.CENTER, Align.CENTER, Align.MIN),
            )
    lego.part.export_svg(
        "tutorial_step11.svg", (-100, -100, 50), (0, 0, 1), svg_opts=svg_opts
    )
    lego.part.export_svg(
        "tutorial_lego.svg",
        (-100, -100, 50),
        (0, 0, 1),
        svg_opts={
            "pixel_scale": 20,
            "show_axes": False,
            "show_hidden": True,
        },
    )

assert abs(lego.part.volume - 3212.187337781355) < 1e-3

if "show_object" in locals():
    show_object(lego.part.wrapped, name="lego")


In [124]:
lego

In [7]:
# %load loft.py
"""

name: loft.py
by:   Gumyr
date: July 15th 2022

desc:

    This example demonstrates lofting a set of sketches, selecting
    the top and bottom by type, and shelling.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from math import pi, sin
from build123d import *

with BuildPart() as art:
    slice_count = 10
    for i in range(slice_count + 1):
        with BuildSketch(Plane(origin=(0, 0, i * 3), z_dir=(0, 0, 1))) as slice:
            Circle(10 * sin(i * pi / slice_count) + 5)
    loft()
    top_bottom = art.faces().filter_by(GeomType.PLANE)
    offset(openings=top_bottom, amount=0.5)

assert abs(art.part.volume - 1306.3405290344635) < 1e-3

if "show_object" in locals():
    show_object(art.part.wrapped, name="art")


In [8]:
art

In [9]:
# %load mixed_algebra_context.py
from build123d import *

# Mix context and algebra api for parts

b = Box(1, 2, 3) + Cylinder(0.75, 2.5)

with BuildPart() as bp:
    add(b)
    Cylinder(0.4, 6, mode=Mode.SUBTRACT)

c = bp.part - Plane.YZ * Cylinder(0.2, 6)

# Mix context and algebra api for sketches

r = Rectangle(1, 2) + Circle(0.75)

with BuildSketch() as bs:
    add(r)
    Circle(0.4, mode=Mode.SUBTRACT)

d = bs.sketch - Pos(0, 1) * Circle(0.2)

# Mix context and algebra api for sketches

l1 = Line((-1, 0), (1, 1)) + Line((1, 1), (2, 4))

with BuildLine() as bl:
    add(l1)
    Line((2, 4), (-1, 1))

e = bl.line + ThreePointArc((-1, 0), (-1.5, 0.5), (-1, 1))

if "show_object" in locals():
    show_object(Pos(0, -2, 0) * c, "part")
    show_object(Pos(0, 2, 0) * d, "sketch")
    show_object(Pos(0, 0, 2) * e, "curve")


In [13]:
c

In [129]:
# %load multiple_workplanes.py
from build123d import *

with BuildPart() as obj:
    Box(5, 5, 1)
    with BuildPart(*obj.faces().filter_by(Axis.Z), mode=Mode.SUBTRACT):
        Sphere(1.8)

assert abs(obj.part.volume - 15.083039190168236) < 1e-3

if "show_object" in locals():
    show_object(obj.part)


In [14]:
# %load pegboard_j_hook.py
"""
name: pegboard_j_hook.py
by:   jdegenstein
date: November 17th 2022
desc:
    This example creates a model of j-shaped pegboard hook commonly used
    for organization of tools in garages.

license:
    Copyright 2022 jdegenstein
    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at
        http://www.apache.org/licenses/LICENSE-2.0
    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""

from build123d import *

pegd = 6.35 + 0.1  # mm ~0.25inch
c2c = 25.4  # mm 1.0inch
arcd = 7.2
both = 10
topx = 6
midx = 8
maind = 0.82 * pegd
midd = 1.0 * pegd
hookd = 23
hookx = 10
splitz = maind / 2 - 0.1
topangs = 70

with BuildPart() as mainp:
    with BuildLine(mode=Mode.PRIVATE) as sprof:
        l1 = Line((-both, 0), (c2c - arcd / 2 - 0.5, 0))
        l2 = JernArc(start=l1 @ 1, tangent=l1 % 1, radius=arcd / 2, arc_size=topangs)
        l3 = PolarLine(
            start=l2 @ 1,
            length=topx,
            direction=l2 % 1,
        )
        l4 = JernArc(start=l3 @ 1, tangent=l3 % 1, radius=arcd / 2, arc_size=-topangs)
        l5 = PolarLine(
            start=l4 @ 1,
            length=topx,
            direction=l4 % 1,
        )
        l6 = JernArc(
            start=l1 @ 0, tangent=(l1 % 0).reverse(), radius=hookd / 2, arc_size=170
        )
        l7 = PolarLine(
            start=l6 @ 1,
            length=hookx,
            direction=l6 % 1,
        )
    with BuildSketch(Plane.YZ):
        Circle(radius=maind / 2)
    sweep(path=sprof.wires()[0])
    with BuildLine(mode=Mode.PRIVATE) as stub:
        l7 = Line((0, 0), (0, midx + maind / 2))
    with BuildSketch(Plane.XZ):
        Circle(radius=midd / 2)
    sweep(path=stub.wires()[0])
    # splits help keep the object 3d printable by reducing overhang
    split(bisect_by=Plane(origin=(0, 0, -splitz)))
    split(bisect_by=Plane(origin=(0, 0, splitz)), keep=Keep.BOTTOM)

if "show_object" in locals():
    show_object(mainp.part.wrapped)


In [15]:
mainp

In [132]:
# %load pillow_block.py
"""

name: pillow_block.py
by:   Gumyr
date: July 14th 2022

desc:

    This example demonstrates placing holes in a part in a rectangular
    array.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

height, width, thickness, padding = 60, 80, 10, 12
screw_shaft_radius, screw_head_radius, screw_head_height = 1.5, 3, 3
bearing_axle_radius, bearing_radius, bearing_thickness = 4, 11, 7

# Build pillow block as an extruded sketch with counter bore holes
with BuildPart() as pillow_block:
    with BuildSketch() as plan:
        Rectangle(width, height)
        fillet(plan.vertices(), radius=5)
    extrude(amount=thickness)
    # with Locations((0, 0, thickness)):
    with Locations(pillow_block.faces().sort_by(Axis.Z)[-1]):
        CounterBoreHole(bearing_axle_radius, bearing_radius, bearing_thickness)
        with GridLocations(width - 2 * padding, height - 2 * padding, 2, 2):
            CounterBoreHole(screw_shaft_radius, screw_head_radius, screw_head_height)

# Render the part
if "show_object" in locals():
    show_object(pillow_block.part.wrapped)


In [133]:
pillow_block

In [134]:
# %load playing_cards.py
"""

name: custom_sketch_objects.py
by:   Gumyr
date: Jan 21st 2023

desc:

    This example demonstrates user generated custom BuildSketch objects.

license:

    Copyright 2023 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from typing import Literal
from build123d import *


# [Club]
class Club(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch() as club:
            with BuildLine():
                l0 = Line((0, -188), (76, -188))
                b0 = Bezier(l0 @ 1, (61, -185), (33, -173), (17, -81))
                b1 = Bezier(b0 @ 1, (49, -128), (146, -145), (167, -67))
                b2 = Bezier(b1 @ 1, (187, 9), (94, 52), (32, 18))
                b3 = Bezier(b2 @ 1, (92, 57), (113, 188), (0, 188))
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / club.sketch.bounding_box().size.Y)
        super().__init__(obj=club.sketch, rotation=rotation, align=align, mode=mode)


# [Club]


class Spade(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch() as spade:
            with BuildLine():
                b0 = Bezier((0, 198), (6, 190), (41, 127), (112, 61))
                b1 = Bezier(b0 @ 1, (242, -72), (114, -168), (11, -105))
                b2 = Bezier(b1 @ 1, (31, -174), (42, -179), (53, -198))
                l0 = Line(b2 @ 1, (0, -198))
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / spade.sketch.bounding_box().size.Y)
        super().__init__(obj=spade.sketch, rotation=rotation, align=align, mode=mode)


class Heart(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch() as heart:
            with BuildLine():
                b1 = Bezier((0, 146), (20, 169), (67, 198), (97, 198))
                b2 = Bezier(b1 @ 1, (125, 198), (151, 186), (168, 167))
                b3 = Bezier(b2 @ 1, (197, 133), (194, 88), (158, 31))
                b4 = Bezier(b3 @ 1, (126, -13), (94, -48), (62, -95))
                b5 = Bezier(b4 @ 1, (40, -128), (0, -198))
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / heart.sketch.bounding_box().size.Y)
        super().__init__(obj=heart.sketch, rotation=rotation, align=align, mode=mode)


class Diamond(BaseSketchObject):
    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch() as diamond:
            with BuildLine():
                Bezier((135, 0), (94, 69), (47, 134), (0, 198))
                mirror(about=Plane.XZ)
                mirror(about=Plane.YZ)
            make_face()
            scale(by=height / diamond.sketch.bounding_box().size.Y)
        super().__init__(obj=diamond.sketch, rotation=rotation, align=align, mode=mode)


card_width = 2.5 * IN
card_length = 3.5 * IN
deck = 0.5 * IN
wall = 4 * MM
gap = 0.5 * MM

with BuildPart() as box_builder:
    with BuildSketch() as plan:
        Rectangle(card_width + 2 * wall, card_length + 2 * wall)
        fillet(plan.vertices(), radius=card_width / 15)
    extrude(amount=wall / 2)
    with BuildSketch(box_builder.faces().sort_by(Axis.Z)[-1]) as walls:
        add(plan.sketch)
        offset(plan.sketch, amount=-wall, mode=Mode.SUBTRACT)
    extrude(amount=deck / 2)
    with BuildSketch(box_builder.faces().sort_by(Axis.Z)[-1]) as inset_walls:
        offset(plan.sketch, amount=-(wall + gap) / 2, mode=Mode.ADD)
        offset(plan.sketch, amount=-wall, mode=Mode.SUBTRACT)
    extrude(amount=deck / 2)

with BuildPart() as lid_builder:
    with BuildSketch() as outset_walls:
        add(plan.sketch)
        offset(plan.sketch, amount=-(wall - gap) / 2, mode=Mode.SUBTRACT)
    extrude(amount=deck / 2)
    with BuildSketch(lid_builder.faces().sort_by(Axis.Z)[-1]) as top:
        add(plan.sketch)
    extrude(amount=wall / 2)
    with BuildSketch(lid_builder.faces().sort_by(Axis.Z)[-1]):
        holes = GridLocations(
            3 * card_width / 5, 3 * card_length / 5, 2, 2
        ).local_locations
        for i, hole in enumerate(holes):
            with Locations(hole) as hole_loc:
                if i == 0:
                    Heart(card_length / 5)
                elif i == 1:
                    Diamond(card_length / 5)
                elif i == 2:
                    Spade(card_length / 5)
                elif i == 3:
                    Club(card_length / 5)
    extrude(amount=-wall, mode=Mode.SUBTRACT)

box = Compound.make_compound(
    [box_builder.part, lid_builder.part.moved(Location((0, 0, (wall + deck) / 2)))]
)
box.export_svg(
    "../docs/assets/card_box.svg",
    (70, -50, 120),
    (0, 0, 1),
    svg_opts={"pixel_scale": 5, "show_axes": False, "show_hidden": False},
)


class PlayingCard(Compound):
    """PlayingCard

    A standard playing card modelled as a Face.

    Args:
        rank (Literal['A', '2' .. '9', 'J', 'Q', 'K']): card rank
        suit (Literal['Clubs', 'Spades', 'Hearts', 'Diamonds']): card suit
    """

    width = 2.5 * IN
    height = 3.5 * IN
    suits = {"Clubs": Club, "Spades": Spade, "Hearts": Heart, "Diamonds": Diamond}
    ranks = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "J", "Q", "K"]

    def __init__(
        self,
        rank: Literal["A", "2", "3", "4", "5", "6", "7", "8", "9", "J", "Q", "K"],
        suit: Literal["Clubs", "Spades", "Hearts", "Diamonds"],
    ):
        with BuildSketch() as playing_card:
            Rectangle(
                PlayingCard.width, PlayingCard.height, align=(Align.MIN, Align.MIN)
            )
            fillet(playing_card.vertices(), radius=PlayingCard.width / 15)
            with Locations(
                (
                    PlayingCard.width / 7,
                    8 * PlayingCard.height / 9,
                )
            ):
                Text(
                    txt=rank,
                    font_size=PlayingCard.width / 7,
                    mode=Mode.SUBTRACT,
                )
            with Locations(
                (
                    PlayingCard.width / 7,
                    7 * PlayingCard.height / 9,
                )
            ):
                PlayingCard.suits[suit](
                    height=PlayingCard.width / 12, mode=Mode.SUBTRACT
                )
            with Locations(
                (
                    6 * PlayingCard.width / 7,
                    1 * PlayingCard.height / 9,
                )
            ):
                Text(
                    txt=rank,
                    font_size=PlayingCard.width / 7,
                    rotation=180,
                    mode=Mode.SUBTRACT,
                )
            with Locations(
                (
                    6 * PlayingCard.width / 7,
                    2 * PlayingCard.height / 9,
                )
            ):
                PlayingCard.suits[suit](
                    height=PlayingCard.width / 12, rotation=180, mode=Mode.SUBTRACT
                )
            rank_int = PlayingCard.ranks.index(rank) + 1
            rank_int = rank_int if rank_int < 10 else 1
            with Locations((PlayingCard.width / 2, PlayingCard.height / 2)):
                center_radius = 0 if rank_int == 1 else PlayingCard.width / 3.5
                suit_rotation = 0 if rank_int == 1 else -90
                suit_height = (
                    0.00159 * rank_int**2 - 0.0380 * rank_int + 0.37
                ) * PlayingCard.width
                with PolarLocations(
                    radius=center_radius,
                    count=rank_int,
                    start_angle=90 if rank_int > 1 else 0,
                ):
                    PlayingCard.suits[suit](
                        height=suit_height,
                        rotation=suit_rotation,
                        mode=Mode.SUBTRACT,
                    )
        super().__init__(playing_card.sketch.wrapped)


playing_card = PlayingCard(rank="A", suit="Spades")

if "show_object" in locals():
    show_object(playing_card)
    # show_object(outer_box_builder.part, "outer")
    # show_object(b, name="b", options={"alpha": 0.8})
    show_object(box_builder.part, "box_builder")
    show_object(
        lid_builder.part.moved(Location((0, 0, (wall + deck) / 2))),
        "lid_builder",
        options={"alpha": 0.7},
    )
    # show_object(walls.sketch, "walls")
    # show_object(o, "o")
    # show_object(half_club.line)
    # show_object(spade_outline.line)
    # show_object(b0, "b0")
    # show_object(b1, "b1")
    # show_object(b2, "b2")
    # show_object(b3, "b3")
    # show_object(b4, "b4")
    # show_object(b5, "b5")
    # show_object(l0, "l0")
    # show_object(l0, "l0")


In [135]:
playing_card = PlayingCard(rank="Q", suit="Clubs")
playing_card

In [136]:
# %load projection.py
"""

Projection Examples

name: projection.py
by:   Gumyr
date: January 4th 2023

desc: Projection examples.

license:

    Copyright 2023 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.

"""
from build123d import *

# A sphere used as a projection target
sphere = Solid.make_sphere(50, angle1=-90)

"""Example 1 - Mapping A Face on Sphere"""
projection_direction = Vector(0, 1, 0)

square = Face.make_rect(20, 20, Plane.ZX.offset(-80))
square_projected = square.project_to_shape(sphere, projection_direction)
square_solids = Compound.make_compound([f.thicken(2) for f in square_projected])
projection_beams = [
    Solid.make_loft(
        [
            square.outer_wire(),
            square.outer_wire().translate(Vector(0, 160, 0)),
        ]
    )
]

"""Example 2 - Flat Projection of Text on Sphere"""
projection_direction = Vector(0, -1, 0)
flat_planar_text_faces = (
    Compound.make_text("Flat", font_size=30).rotate(Axis.X, 90).faces()
)
flat_projected_text_faces = Compound.make_compound(
    [
        f.project_to_shape(sphere, projection_direction)[0]
        for f in flat_planar_text_faces
    ]
).moved(Location((-100, -100)))
flat_projection_beams = Compound.make_compound(
    [Solid.extrude_linear(f, projection_direction * 80) for f in flat_planar_text_faces]
).moved(Location((-100, -100)))


"""Example 3 - Project a text string along a path onto a shape"""
arch_path: Edge = (
    sphere.cut(Solid.make_cylinder(80, 100, Plane.YZ).locate(Location((-50, 0, -70))))
    .edges()
    .sort_by(Axis.Z)[0]
)
arch_path_start = Vertex(arch_path.position_at(0))
text = Compound.make_text(
    txt="'the quick brown fox jumped over the lazy dog'",
    font_size=15,
    align=(Align.MIN, Align.CENTER),
)
projected_text = sphere.project_faces(text, path=arch_path)

if "show_object" in locals():
    # Example 1
    show_object(sphere, name="sphere_solid", options={"alpha": 0.8})
    show_object(square, name="square")
    show_object(square_solids, name="square_solids")
    show_object(
        Compound.make_compound(projection_beams),
        name="projection_beams",
        options={"alpha": 0.9, "color": (170 / 255, 170 / 255, 255 / 255)},
    )

    # Example 2
    show_object(
        sphere.moved(Location((-100, -100))),
        name="sphere_solid for text",
        options={"alpha": 0.8},
    )
    show_object(flat_projected_text_faces, name="flat_projected_text_faces")
    show_object(
        flat_projection_beams,
        name="flat_projection_beams",
        options={"alpha": 0.95, "color": (170 / 255, 170 / 255, 255 / 255)},
    )

    # Example 3
    show_object(
        sphere.moved(Location((100, 100))),
        name="sphere_solid for text on path",
        options={"alpha": 0.8},
    )
    show_object(
        projected_text.moved(Location((100, 100))), name="projected_text on path"
    )


In [137]:
projected_text

In [138]:
# %load roller_coaster.py
"""

name: roller_coaster.py
by:   Gumyr
date: July 19th 2022

desc:

    This example demonstrates building complex 3D lines by "snapping"
    features to existing objects.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

with BuildLine() as roller_coaster:
    powerup = Spline(
        (0, 0, 0),
        (50, 0, 50),
        (100, 0, 0),
        tangents=((1, 0, 0), (1, 0, 0)),
        tangent_scalars=(0.5, 2),
    )
    corner = RadiusArc(powerup @ 1, (100, 60, 0), -30)
    screw = Helix(75, 150, 15, center=(75, 40, 15), direction=(-1, 0, 0))
    Spline(corner @ 1, screw @ 0, tangents=(corner % 1, screw % 0))
    Spline(screw @ 1, (-100, 30, 10), powerup @ 0, tangents=(screw % 1, powerup % 0))

if "show_object" in locals():
    show_object(roller_coaster.line.wrapped, name="roller_coaster")


In [139]:
roller_coaster

In [140]:
# %load shamrock.py
from build123d import *


class Shamrock(BaseSketchObject):
    """Sketch Object: Shamrock

    Adds a four leaf clover

    Args:
        height (float): y axis dimension
        rotation (float, optional): angle in degrees. Defaults to 0.
        align (tuple[Align, Align], optional): alignment. Defaults to (Align.CENTER, Align.CENTER).
        mode (Mode, optional): combination mode. Defaults to Mode.ADD.
    """

    def __init__(
        self,
        height: float,
        rotation: float = 0,
        align: tuple[Align, Align] = (Align.CENTER, Align.CENTER),
        mode: Mode = Mode.ADD,
    ):
        with BuildSketch() as shamrock:
            with BuildLine():
                b0 = Bezier((240, 310), (112, 325), (162, 438), (252, 470))
                b1 = Bezier(b0 @ 1, (136, 431), (73, 589), (179, 643))
                b2 = Bezier(b1 @ 1, (151, 747), (293, 770), (360, 679))
                b3 = Bezier(b2 @ 1, (358, 736), (366, 789), (392, 840))
                l0 = Line(b3 @ 1, (420, 820))
                b4 = Bezier(l0 @ 1, (366, 781), (374, 670), (380, 670))
                b5 = Bezier(b4 @ 1, (400, 794), (506, 789), (528, 727))
                b6 = Bezier(b5 @ 1, (636, 733), (638, 578), (507, 541))
                b7 = Bezier(b6 @ 1, (628, 559), (651, 380), (575, 365))
                b8 = Bezier(b7 @ 1, (592, 269), (420, 268), (417, 361))
                b9 = Bezier(b8 @ 1, (410, 253), (262, 222), b0 @ 0)
                mirror(about=Plane.XZ, mode=Mode.REPLACE)
            make_face()
            scale(by=height / shamrock.sketch.bounding_box().size.Y)
        super().__init__(
            obj=shamrock.sketch.translate(
                -shamrock.sketch.center(CenterOf.BOUNDING_BOX)
            ),
            rotation=rotation,
            align=align,
            mode=mode,
        )


if __name__ == "__main__" or "show_object" in locals():
    with BuildSketch() as shamrock_example:
        Shamrock(10)

    if "show_object" in locals():
        show_object(shamrock_example.sketch)


In [141]:
with BuildSketch() as shamrock_example:
    Shamrock(10)
    
shamrock_example

In [142]:
# %load tea_cup.py
"""

name: tea_cup.py
by:   Gumyr
date: March 27th 2023

desc: This example demonstrates the creation of non-planar objects.

license:

    Copyright 2023 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

wall_thickness = 3 * MM
fillet_radius = wall_thickness * 0.49

with BuildPart() as tea_cup:
    # Create the bowl of the cup as a revolved cross section
    with BuildSketch(Plane.XZ) as bowl_section:
        with BuildLine():
            # Start & end points with control tangents
            s = Spline(
                (30 * MM, 10 * MM),
                (69 * MM, 105 * MM),
                tangents=((1, 0.5), (0.7, 1)),
                tangent_scalars=(1.75, 1),
            )
            # Lines to finish creating ½ the bowl shape
            Polyline(s @ 0, s @ 0 + (10 * MM, -10 * MM), (0, 0), (0, (s @ 1).Y), s @ 1)
        make_face()  # Create a filled 2D shape
    revolve(axis=Axis.Z)
    # Hollow out the bowl with openings on the top and bottom
    offset(amount=-wall_thickness, openings=tea_cup.faces().filter_by(GeomType.PLANE))
    # Add a bottom to the bowl
    with Locations((0, 0, (s @ 0).Y)):
        Cylinder(radius=(s @ 0).X, height=wall_thickness)
    # Smooth out all the edges
    fillet(tea_cup.edges(), radius=fillet_radius)

    # Determine where the handle contacts the bowl
    handle_intersections = [
        tea_cup.part.find_intersection(
            Axis(origin=(0, 0, vertical_offset), direction=(1, 0, 0))
        )[-1][0]
        for vertical_offset in [35 * MM, 80 * MM]
    ]
    # Create a path for handle creation
    with BuildLine(Plane.XZ) as handle_path:
        path_spline = Spline(
            handle_intersections[0] - (wall_thickness / 2, 0),
            handle_intersections[0] + (35 * MM, 30 * MM),
            handle_intersections[0] + (40 * MM, 60 * MM),
            handle_intersections[1] - (wall_thickness / 2, 0),
            tangents=((1, 1.25), (-0.2, -1)),
        )
    # Align the cross section to the beginning of the path
    with BuildSketch(
        Plane(origin=path_spline @ 0, z_dir=path_spline % 0)
    ) as handle_cross_section:
        RectangleRounded(wall_thickness, 8 * MM, fillet_radius)
    sweep()  # Sweep handle cross section along path

assert abs(tea_cup.part.volume - 130326) < 1

if "show_object" in locals():
    show_object(tea_cup.part, name="tea cup")


In [143]:
tea_cup

In [144]:
# %load vase.py
"""

name: vase.py
by:   Gumyr
date: July 15th 2022

desc:

    This example demonstrates revolving a sketch, shelling and selecting edges
    by position range and type for fillets.

license:

    Copyright 2022 Gumyr

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
"""
from build123d import *

with BuildPart() as vase:
    with BuildSketch() as profile:
        with BuildLine() as outline:
            l1 = Line((0, 0), (12, 0))
            l2 = RadiusArc(l1 @ 1, (15, 20), 50)
            l3 = Spline(l2 @ 1, (22, 40), (20, 50), tangents=(l2 % 1, (-0.75, 1)))
            l4 = RadiusArc(l3 @ 1, l3 @ 1 + Vector(0, 5), 5)
            l5 = Spline(
                l4 @ 1,
                l4 @ 1 + Vector(2.5, 2.5),
                l4 @ 1 + Vector(0, 5),
                tangents=(l4 % 1, (-1, 0)),
            )
            Polyline(
                l5 @ 1,
                l5 @ 1 + Vector(0, 1),
                (0, (l5 @ 1).Y + 1),
                l1 @ 0,
            )
        make_face()
    revolve(axis=Axis.Y)
    offset(openings=vase.faces().filter_by(Axis.Y)[-1], amount=-1)
    top_edges = (
        vase.edges().filter_by_position(Axis.Y, 60, 62).filter_by(GeomType.CIRCLE)
    )
    fillet(top_edges, radius=0.25)
    fillet(vase.edges().sort_by(Axis.Y)[0], radius=0.5)


if "show_object" in locals():
    # show_object(outline.line.wrapped, name="outline")
    # show_object(profile.sketch.wrapped, name="profile")
    show_object(vase.part.wrapped, name="vase")


In [145]:
vase